#Group D.A.305 by Jorge Montalvo and Laura Cerpa


#URL JSON file from 311 data open source.
https://gis-mdc.opendata.arcgis.com/datasets/MDC::311-service-requests-miami-dade-county-2023/api

#### This data pertains to all service records collected by 311 service during the year 2023. The original CSV file presents inconsistency in the data structure and cleaning through excel would have taken time away from this project.

#### Our first step is to import request library to access the API/JSON

In [148]:
import requests
import pandas
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split



# API URL
url = "https://services.arcgis.com/8Pc9XBTAsYuxx9Ny/arcgis/rest/services/data_311_2023/FeatureServer/0/query?where=1%3D1&outFields=ticket_id,issue_type,case_owner_description,street_address,city,zip_code,neighborhood_district,ticket_created_date_time,ticket__last_update_date_time,ticket_closed_date_time,ticket_status,location_city,sr_xcoordinate,sr_ycoordinate,latitude,longitude,method_received,sr_priority,actual_completed_days&outSR=4326&f=json"

# Make a GET request to the API
response = requests.get(url)

if response.status_code == 200:
    data = response.json()  # Parse the response JSON
else:
    print("Error:", response.status_code)


In [96]:
# Check the keys to understand the data structure
data.keys()  # See what top-level keys are available

# If the data is nested, access the relevant key (like 'features')
records = data.get('features', [])


In [97]:
# Ensure all dictionaries have the same keys
unique_keys = set()
for record in records:
    unique_keys.update(record.keys())

# Fill missing keys with default values to ensure consistency
for record in records:
    for key in unique_keys:
        if key not in record:
            record[key] = None  # Or another default value


In [98]:
df.fillna(value='N/A', inplace=True)  # Fill missing values with a placeholder


In [99]:
df = pd.DataFrame(records)  # Convert the cleaned data to a attribute
df.head()  # Display the first few rows to confirm structure


,attributes
0,"{'ticket_id': '23-10000374', 'issue_type': 'BU..."
1,"{'ticket_id': '23-10000376', 'issue_type': 'BR..."
2,"{'ticket_id': '23-10000377', 'issue_type': 'IN..."
3,"{'ticket_id': '23-10000378', 'issue_type': 'BU..."
4,"{'ticket_id': '23-10000379', 'issue_type': 'IL..."


### The data was flattend into attributes, now it must be converted into Data Frames

In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   attributes  1000 non-null   object
dtypes: object(1)
memory usage: 7.9+ KB


In [101]:

# Convert the string representation of the object into a dictionary
df['attributes'] = df['attributes'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)

# Flatten the nested data into separate columns
df_flat = df['attributes'].apply(pd.Series)  # Convert nested data into a DataFrame
df_flat.head()  # Display the first few rows of the flattened DataFrame
df_311 = df_flat



In [102]:
df_311.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ticket_id                      1000 non-null   object 
 1   issue_type                     1000 non-null   object 
 2   case_owner_description         1000 non-null   object 
 3   street_address                 999 non-null    object 
 4   city                           999 non-null    object 
 5   zip_code                       999 non-null    float64
 6   neighborhood_district          1000 non-null   object 
 7   ticket_created_date_time       1000 non-null   int64  
 8   ticket__last_update_date_time  1000 non-null   int64  
 9   ticket_closed_date_time        993 non-null    float64
 10  ticket_status                  1000 non-null   object 
 11  location_city                  999 non-null    object 
 12  sr_xcoordinate                 989 non-null    fl

#We will apply some Data Cleaning techniques prior to the analysis.:)

In [103]:
# Remove the "location_city" column from df
df_311.drop(columns=['location_city'], inplace=True)  # inplace=True modifies the DataFrame directly


In [104]:
df_311.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ticket_id                      1000 non-null   object 
 1   issue_type                     1000 non-null   object 
 2   case_owner_description         1000 non-null   object 
 3   street_address                 999 non-null    object 
 4   city                           999 non-null    object 
 5   zip_code                       999 non-null    float64
 6   neighborhood_district          1000 non-null   object 
 7   ticket_created_date_time       1000 non-null   int64  
 8   ticket__last_update_date_time  1000 non-null   int64  
 9   ticket_closed_date_time        993 non-null    float64
 10  ticket_status                  1000 non-null   object 
 11  sr_xcoordinate                 989 non-null    float64
 12  sr_ycoordinate                 989 non-null    fl

In [105]:
df_311.columns = df_311.columns.str.strip()  # Remove leading/trailing spaces
df_311.columns = df_311.columns.str.title()  # Convert to Title Case
df_311.head()


,Ticket_Id,Issue_Type,Case_Owner_Description,Street_Address,City,Zip_Code,Neighborhood_District,Ticket_Created_Date_Time,Ticket__Last_Update_Date_Time,Ticket_Closed_Date_Time,Ticket_Status,Sr_Xcoordinate,Sr_Ycoordinate,Latitude,Longitude,Method_Received,Sr_Priority,Actual_Completed_Days
0,23-10000374,BULKY TRASH REQUEST BY APPOINTMENT,Solid Waste Management,21122 SW 92ND CT,Town_of_Cutler_Bay,33189.0,District 8,1672679587632,1673645222313,1.673645e+12,CLOSED,873554.351,450424.639,25.571308,-80.340525,WEB,STANDARD,11.0
1,23-10000376,BREEDER / PET STORE FOLLOW-UP,Enforcement Section,9690 NW 41ST ST,City_of_Doral,33178.0,District 12,1672679829476,1673196028842,1.672759e+12,CLOSED,869217.668,537584.781,25.811171,-80.352385,INHOUSE,STANDARD,1.0
2,23-10000377,INJURED ANIMAL,Enforcement Section,NW 142ND LN & NW 87TH AVE,City_of_Miami_Lakes,33018.0,District 13,1672679862513,1672684925286,1.672685e+12,CLOSED,872951.375,570986.622,25.903017,-80.340527,INHOUSE,EMERGNCY,0.0
3,23-10000378,BULKY TRASH REQUEST BY APPOINTMENT,Solid Waste Management,10380 SW 66TH TER,Miami_Dade_County,33173.0,District 10,1672679952175,1675115223981,1.675115e+12,CLOSED,866149.750,499350.780,25.706020,-80.362272,WEB,STANDARD,28.0
4,23-10000379,ILLEGAL DUMPING / LITTER,COM Solid Waste,1656 SW 6TH ST,City_of_Miami,33135.0,District 5,1672679982339,1674068754054,1.674069e+12,CLOSED,912160.747,521835.219,25.767201,-80.222142,IPHONE,STANDARD,16.0


In [130]:
# Rename columns
df_311.rename(columns={'Ticket__Last_Update_Date_Time': 'Ticket_Last_Update_Date_Time'}, inplace=True)  # Use inplace to modify the original DataFrame
df_311.rename(columns={'Sr_Xcoordinate': 'X_coordinate'}, inplace=True)
df_311.rename(columns={'Sr_Ycoordinate': 'Y_coordinate'}, inplace=True)

#### **After the data was split in df_311 some of the columns have inconsistencies in data type and structure. The following columns need to be converted to strings and reformat to date and time:**
ticket_created_date_time       int64  
ticket_last_update_date_time  int64  
ticket_closed_date_time        float64

In [136]:
df_311.columns  #all column names


Index(['Ticket_Id', 'Issue_Type', 'Case_Owner_Description', 'Street_Address',
       'City', 'Zip_Code', 'Neighborhood_District', 'Ticket_Created_Date_Time',
       'Ticket_Last_Update_Date_Time', 'Ticket_Closed_Date_Time',
       'Ticket_Status', 'X_coordinate', 'Y_coordinate', 'Latitude',
       'Longitude', 'Method_Received', 'Sr_Priority', 'Actual_Completed_Days'],
      dtype='object')

In [141]:
df_311[['Ticket_Created_Date_Time', 'Ticket_Last_Update_Date_Time', 'Ticket_Closed_Date_Time']].info()  #to understand the structure
df_311[['Ticket_Created_Date_Time', 'Ticket_Last_Update_Date_Time', 'Ticket_Closed_Date_Time']].head()  # Inspect the data


<class 'pandas.core.frame.DataFrame'>
Index: 981 entries, 0 to 999
Data columns (total 3 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Ticket_Created_Date_Time      981 non-null    int64  
 1   Ticket_Last_Update_Date_Time  981 non-null    int64  
 2   Ticket_Closed_Date_Time       981 non-null    float64
dtypes: float64(1), int64(2)
memory usage: 30.7 KB


,Ticket_Created_Date_Time,Ticket_Last_Update_Date_Time,Ticket_Closed_Date_Time
0,1672679587632,1673645222313,1.673645e+12
1,1672679829476,1673196028842,1.672759e+12
2,1672679862513,1672684925286,1.672685e+12
3,1672679952175,1675115223981,1.675115e+12
4,1672679982339,1674068754054,1.674069e+12


##Data Cleaning
### In order to convert the data to date stamp we need to eliminate empty entries and add a zero value

In [143]:
df_311.isnull().sum()  # Get the count of null values in each column


Ticket_Id                       0
Issue_Type                      0
Case_Owner_Description          0
Street_Address                  0
City                            0
Zip_Code                        0
Neighborhood_District           0
Ticket_Created_Date_Time        0
Ticket_Last_Update_Date_Time    0
Ticket_Closed_Date_Time         0
Ticket_Status                   0
X_coordinate                    0
Y_coordinate                    0
Latitude                        0
Longitude                       0
Method_Received                 0
Sr_Priority                     0
Actual_Completed_Days           0
dtype: int64

In [146]:
import pandas as pd
# Convert from milliseconds to datetime, setting unit='ms'
df_311['Ticket_Created_Date_Time'] = pd.to_datetime(df_311['Ticket_Created_Date_Time'], unit='ms')
df_311['Ticket_Last_Update_Date_Time'] = pd.to_datetime(df_311['Ticket_Last_Update_Date_Time'], unit='ms')
df_311['Ticket_Closed_Date_Time'] = pd.to_datetime(df_311['Ticket_Closed_Date_Time'], unit='ms')


In [147]:
df_311[['Ticket_Created_Date_Time', 'Ticket_Last_Update_Date_Time', 'Ticket_Closed_Date_Time']].head()  # Inspect the data

,Ticket_Created_Date_Time,Ticket_Last_Update_Date_Time,Ticket_Closed_Date_Time
0,2023-01-02 17:13:07.632,2023-01-13 21:27:02.313,2023-01-13 21:25:43.000000000
1,2023-01-02 17:17:09.476,2023-01-08 16:40:28.842,2023-01-03 15:15:28.820000000
2,2023-01-02 17:17:42.513,2023-01-02 18:42:05.286,2023-01-02 18:42:05.286000128
3,2023-01-02 17:19:12.175,2023-01-30 21:47:03.981,2023-01-30 21:45:48.000000000
4,2023-01-02 17:19:42.339,2023-01-18 19:05:54.054,2023-01-18 19:05:54.054000128


In [149]:
df_311[['Ticket_Created_Date_Time', 'Ticket_Last_Update_Date_Time', 'Ticket_Closed_Date_Time']].tail()  # Inspect the data

,Ticket_Created_Date_Time,Ticket_Last_Update_Date_Time,Ticket_Closed_Date_Time
994,2023-01-03 14:45:08.272,2023-03-27 04:00:00.000,2023-03-27 04:00:00.000
995,2023-01-03 15:16:06.772,2023-01-09 05:01:00.000,2023-01-09 05:00:00.000
997,2023-01-03 15:16:23.559,2023-01-13 19:22:03.699,2023-01-13 19:20:27.000
998,2023-01-03 15:17:09.611,2023-01-17 21:29:18.172,2023-01-17 21:29:18.172
999,2023-01-03 15:17:28.036,2023-01-18 15:02:02.051,2023-01-18 15:00:39.000


In [150]:
# Filter and group by issue type, then count and order by descending
common_service_requests = df_311[df_311['Ticket_Status'] == 'CLOSED'] \
    .groupby('Issue_Type') \
    .size() \
    .sort_values(ascending=False) \
    .head(10)  # Limit to the top 10

common_service_requests


Issue_Type
BULKY TRASH REQUEST BY APPOINTMENT    337
GREEN WASTE CART REQUEST              145
BULKY TRASH COMPLAINT                  57
RECYCLING BLUE CART ISSUES             35
POTHOLE                                31
ABANDONED SHOPPING CARTS               27
ASU POLICE ASSISTANCE                  25
DEAD ANIMAL PICKUP MD                  22
GARBAGE COMPLAINT                      20
RECYCLING COMPLAINT                    17
dtype: int64

In [155]:
# Filter for 'Bulky Trash' and group by zip code and city
top_bulky_trash_zip_codes = df_311[
    (df_311['Ticket_Status'] == 'CLOSED') &
    (df_311['Issue_Type'] == 'BULKY TRASH REQUEST')
] \
.groupby(['Zip_Code']) \
.size() \
.sort_values(ascending=False) \
.head(5)  # Limit to top 5

top_bulky_trash_zip_codes



Series([], dtype: int64)

In [156]:
# Group by service request priority
common_service_priority = df_311[df_311['Ticket_Status'] == 'CLOSED'] \
    .groupby('Sr_Priority') \
    .size() \
    .sort_values(ascending=False)

common_service_priority


Sr_Priority
STANDARD    842
URGENT       33
EMERGNCY     26
dtype: int64

In [158]:
# Group by case owner and count, sorted by descending
department_with_most_requests = df_311[
    df_311['Ticket_Status'] == 'CLOSED'
] \
.groupby('Case_Owner_Description') \
.size() \
.sort_values(ascending=False)

department_with_most_requests


Case_Owner_Description
Solid Waste Management               689
Enforcement Section                   89
Regulatory and Economic Resources     48
Road And Bridges                      33
Citations and Tags                    19
COM Solid Waste                        9
Traffic Operation                      6
RAAM                                   5
Traffic Signals And Signs              3
dtype: int64

In [162]:
# Filter for Solid Waste Management and group by issue type
common_service_in_solid_waste = df_311[
    (df_311['Ticket_Status'] == 'CLOSED') &
    (df_311['Case_Owner_Description'] == 'Solid_Waste_Management')]

common_service_in_solid_waste


,Ticket_Id,Issue_Type,Case_Owner_Description,Street_Address,City,Zip_Code,Neighborhood_District,Ticket_Created_Date_Time,Ticket_Last_Update_Date_Time,Ticket_Closed_Date_Time,Ticket_Status,X_coordinate,Y_coordinate,Latitude,Longitude,Method_Received,Sr_Priority,Actual_Completed_Days


In [163]:
# Group by issue type and case owner to find average completion days
service_requests_with_long_closure = df_311[
    df_311['Ticket_Status'] == 'CLOSED'
] \
.groupby(['Issue_Type', 'Case_Owner_Description']) \
.agg({'Actual_Completed_Days': 'mean'}) \
.round() \
.sort_values(by='Actual_Completed_Days', ascending=False) \
.head(10)

service_requests_with_long_closure


,,Actual_Completed_Days
Issue_Type,Case_Owner_Description,
TREE TRIMMING (RAAM),RAAM,257.0
TNVR ASSISTANCE FOR COMMUNITY CATS,Enforcement Section,218.0
ANIMAL CRUELTY FOLLOW-UP,Enforcement Section,145.0
JUNK AND TRASH / OVERGROWTH,Regulatory and Economic Resources,105.0
TRAFFIC SIGNAL BULB BURNED OUT,Traffic Signals And Signs,80.0
TREE REMOVAL (RAAM),RAAM,59.0
COMMERCIAL VEHICLE - UNAUTHORIZED,Regulatory and Economic Resources,57.0
UNAUTHORIZED USE - RESIDENTIAL / BUSINESS,Regulatory and Economic Resources,49.0
TETHERED DOG FOLLOW-UP,Enforcement Section,38.0


In [166]:
# Group by case owner and find average actual and goal days
department_performance = df_311[
    df_311['Ticket_Status'] == 'CLOSED'
] \
.groupby('Case_Owner_Description') \
.agg({
    'Actual_Completed_Days': 'mean',
    'Ticket_Created_Date_Time': 'mean'  # Assuming this represents goal days
}) \
.round() \
.sort_values(by='Actual_Completed_Days', ascending=True)  # Sort by goal days

department_performance


,Actual_Completed_Days,Ticket_Created_Date_Time
Case_Owner_Description,,
Citations and Tags,3.0,2023-01-03 14:37:40.378684160
Traffic Operation,3.0,2023-01-03 03:25:01.284333312
COM Solid Waste,6.0,2023-01-02 13:07:29.932555520
Solid Waste Management,10.0,2023-01-03 00:02:09.677085440
Enforcement Section,11.0,2023-01-02 19:49:43.159134976
Road And Bridges,20.0,2023-01-03 00:45:02.655242496
Traffic Signals And Signs,27.0,2023-01-03 01:37:11.479333632
Regulatory and Economic Resources,35.0,2023-01-03 00:08:03.941520640
RAAM,167.0,2023-01-03 07:07:25.491800064


In [167]:
# Group by city and count requests, excluding 'Miami_Dade_County'
city_with_highest_requests = df_311[
    (df_311['Ticket_Status'] == 'CLOSED') &
    (df_311['City'] != 'Miami_Dade_County')
] \
.groupby('City') \
.size() \
.sort_values(ascending=False) \
.head(10)

city_with_highest_requests


City
City_of_Miami_Gardens        64
City_of_Doral                30
Town_of_Cutler_Bay           27
City_of_Miami                26
City_of_Pinecrest            16
Village_of_Palmetto_Bay      12
City_of_Miami_Lakes          11
City_of_Hialeah               7
City_of_North_Miami_Beach     5
City_of_Palmetto_Bay          4
dtype: int64

In [168]:
# Group by method received and count requests
top_methods = df_311.groupby('Method_Received') \
    .size() \
    .sort_values(ascending=False) \
    .head(5)  # Limit to top 5

top_methods


Method_Received
WEB         472
PHONE       309
IPHONE       91
INHOUSE      31
XTERFACE     31
dtype: int64